<a href="https://colab.research.google.com/github/agdoko/deep_green_learning/blob/master/Baseline_Model_Notebook%20with%20ARB%20sample2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [4]:
# Required imports
import google
import ee
import folium
import requests
import io
import sklearn
import numpy as np

from google.colab import auth
from folium import plugins
from sklearn.metrics import f1_score


# geemap needs to be installed first in this colab environment
!pip install geemap
import geemap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.7 MB/s eta 0:00:00


# Authentication flow

In [5]:
# Standard authentication cell
auth.authenticate_user()
credentials, project_id = google.auth.default()
ee.Initialize(credentials, project='semiotic-garden-395711')

In [6]:
# Testing the authentication worked:
print(ee.Image("NASA/NASADEM_HGT/001").get("title").getInfo())

NASADEM: NASA NASADEM Digital Elevation 30m


# NEW DEFINING CUSTOM FUNCTIONS

## Add Layer Function

In [ ]:
# Defining the required add.layer function, which is not native to folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

## Clipping Function

In [ ]:
# Defining the clipping feature — this is required because filter by Bounds only
# limits the image collection to images that intersetc the Geometry defined, but
# does not actually cut out the Geometry from the images that intersect
def clip_img(img):
    return img.clip(square)

## Majority Pooling Function

In [ ]:
# Defining the majority pooling function
def majority_pool(array, new_shape=(2, 2, 6)):
    # Initialize the new pooled array
    pooled_array = np.zeros(new_shape, dtype=array.dtype)

    # Calculate the step sizes for x and y dimensions
    step_x = array.shape[0] // new_shape[0]
    step_y = array.shape[1] // new_shape[1]

    # Loop through each dimension
    for z in range(array.shape[2]):
        for new_x in range(new_shape[0]):
            for new_y in range(new_shape[1]):
                x_start = new_x * step_x
                y_start = new_y * step_y
                quadrant = array[x_start:x_start+step_x, y_start:y_start+step_y, z]
                majority = np.sum(quadrant) > (step_x * step_y // 2)
                pooled_array[new_x, new_y, z] = majority

    return pooled_array

# NEW ACCEPTING THE COORDINATES FROM ANA's + FELIX's SAMPLING
*   FELIX PROVIDES A POINT
*   ANA PROVIDES 4 CORNER COORDINATES

In [11]:
# Defining start and end dates. Note the target dataset is annual, while the
# features are updated every 5-10 days

# Initial year of interest (inclusive). Picked because features only go to
# 2015 and start in june, the earliest january images for features is 2016.
i_date = '2016'

# Final year of interest (exclusive). Picked because target only goes to 2021.
# So we should have 6 matches in total between features and targets (whole globe)
f_date = '2022'

In [26]:
# TO DO - need to adjust this to accept either a point from Felix or Ana 4 coordinates

# specify these coordinates, then expand a 500 x 500m square around, somewhere in Amazon rainforest
import random
# Adjust these ranges to suit the geographical area of interest - South America (47.7% of SA is known to be covered by forests, including one of the largest tropical forests in the world in the Amazon area)
min_lat = 55.05 # Southernmost point of SA
max_lat = 12.44  # Northernmost point of SA
min_lon = -81.28 # Westernmost point of SA
max_lon = -34.47  # Easternmost point of SA
num_points = 100
# Generate a list of random latitude and longitude points
coordinates = []
for _ in range(num_points):
    lat = random.uniform(min_lat, max_lat)
    lon = random.uniform(min_lon, max_lon)
    coordinates.append((lon, lat))
# Print the list of coordinates
for i, (lon, lat) in enumerate(coordinates):
    print(f"({lat}, {lon}),")

(39.48079615143671, -50.488463069851164),
(48.37322987135536, -51.049709710299524),
(43.817217711864124, -43.68850667895554),
(14.366835570186865, -80.49104327991654),
(45.50064729434144, -41.17558335601902),
(42.74583975378117, -68.96464322218952),
(33.68350762031534, -78.42876455532598),
(55.03655726710421, -78.51142630836692),
(51.12950957196393, -72.63228205280495),
(38.40485686060909, -40.01163618144849),
(15.021934754467573, -44.698011499637936),
(52.23912819246009, -75.85155560867011),
(45.046335149253224, -61.15086224108893),
(24.288418433659107, -40.36653745410474),
(47.402542105280446, -76.03801903062184),
(26.881078246232857, -47.23130329850124),
(17.72111231181934, -74.25966070220673),
(18.020787375572795, -63.7704589946113),
(47.5791996207507, -63.1199723311251),
(14.527489188877318, -41.577539987794445),
(17.26386522734083, -49.55474682032292),
(52.829610784795115, -67.4004203297585),
(41.112608570613084, -62.651556110021886),
(20.58653191333149, -65.12767247161496),
(42.

In [36]:
# Define a list of center coordinates (lon, lat)
coordinates = [
(13.45555639594199, -54.44045226080955),
(36.14676818709064, -43.180266907414826),
(42.99905927897668, -42.1325837805427),
(54.933825631807, -64.65448650314612),
(47.49660202992008, -56.80269204541623),
(40.99399531328745, -46.70148993251093),
(13.866584859600984, -49.637526391504835),
(17.79789494115967, -45.81381809781966),
(48.60813636362735, -47.70228945170309),
(48.00382070030193, -55.59803109169262),
(35.75155827687374, -35.047994404590746),
(49.158789995205346, -51.433930320631305),
(25.946154632684678, -62.84104862443571),
(26.565389110844492, -77.88580186047754),
(39.360058428530465, -72.15769484099808),
(24.716133762988036, -52.57815995922864),
(34.57825073115286, -47.75602182000067),
(53.58350816350808, -53.806630697756916),
(45.92947709667748, -78.91992883561065),
(46.71786871089591, -58.94520027509928),
(13.962336786433767, -34.48440091132063),
(49.902580208409574, -50.477606924235715),
(14.652436631456268, -52.685056533928574),
(52.78016741031922, -60.7480366939013),
(53.800824008123676, -53.703336725389626),
(43.656959202485695, -39.638335025243),
(25.523623559303108, -74.42311469713007),
(53.984696169828474, -71.38577772965849),
(20.682045626798683, -78.60977508704582),
(52.852512489596776, -61.822531050928674),
(47.14796785389777, -67.32234182000013),
(44.00200653484523, -59.021427965622934),
(48.84135552741881, -46.320245006204715),
(28.532712396246538, -38.317010460100946),
(19.058003462212014, -35.03023298215341),
(54.30442382611483, -37.472424369008245),
(43.776866194014765, -45.65494431422484),
(40.97150614059379, -79.28555443919481),
(53.231068766575085, -67.96151673202777),
(47.4293857969865, -70.6579075965423),
(36.807093424516495, -34.61834711571612),
(27.16448664535777, -66.86426071833914),
(37.59914920974688, -54.68053392631865),
(44.22004795104323, -41.55306448615202),
(36.218151677237046, -77.0379499336405),
(27.17272746727061, -45.84091078142265),
(21.36684205999922, -42.78605069909058),
(16.200846786088, -50.04211327727626),
(39.151637813965884, -63.89631968181023),
(27.39191292161154, -46.88604276049577),
(46.474127634941674, -49.56256388764156),
(28.2570480566787, -57.296358578662954),
(34.341862262567375, -48.68523139423558),
(24.824550835538854, -62.18929095922928),
(14.753677909484061, -42.00418094351177),
(37.28246660029153, -79.79271186889872),
(41.346110718353046, -38.024573153685026),
(33.308072999565525, -58.211428090578394),
(40.40621220357113, -58.946174570075215),
(20.34950567000002, -54.990733519426925),
(44.75201438784121, -62.46561162688802),
(27.798721370065163, -53.95819356882248),
(47.78710954577359, -44.50993576471512),
(42.574339054249414, -36.468637061355025),
(29.426073256924326, -60.97441295019361),
(18.790926291240318, -49.664219670465876),
(20.310663355385586, -73.54757418721594),
(54.951362333902, -75.23715050124744),
(19.40001397620952, -80.05869434864262),
(46.446361866625644, -59.13650036115958),
(16.369633330744463, -61.0604982934516),
(28.488422149920734, -38.78220833470239),
(27.24303639045189, -64.00950248561065),
(24.122905484222834, -41.76843913760783),
(17.74367943910395, -61.32249785325798),
(37.146605843838444, -54.060351815816965),
(32.93253322978716, -70.27375530450612),
(51.017108529302554, -37.16491827786441),
(51.74453082112995, -42.622564319333414),
(29.676813460931786, -40.90200648495829),
(48.53136931293487, -67.00493800505946),
(36.77980080252874, -63.27544659831628),
(12.598479290531039, -50.9214710611715),
(52.58188773216458, -46.321617720441644),
(32.418997008786135, -39.52860036862937),
(50.85651650164405, -72.66957869018202),
(18.574122246741688, -39.57891130320519),
(40.840751193253475, -49.74939707897209),
(32.85074528622633, -73.37740476129977),
(16.59520024565238, -36.09426102988652),
(17.29830923779373, -68.78220862503848),
(20.841918956138713, -61.46758821854582),
(18.49408041855017, -56.666925910773614),
(20.783145154175664, -64.01813476891815),
(30.268005639970955, -57.590932277802075),
(46.83707155956431, -64.76968903086909),
(24.163995706207654, -39.955442321546094),
(33.211895423262284, -68.09182621819959),
(49.45687230692794, -73.50679745556874),
(26.928470166878164, -70.75091929611466)
]

# Define the half-width and half-height of the rectangle (in meters)
half_width = 11.15  # Half of 22.3 meters
half_height = 11.15  # Half of 22.3 meters

# Create a list to store the rectangle data as dictionaries
rectangle_data_list = []

# Loop through the list of center coordinates
for i, (lon, lat) in enumerate(coordinates, start=1):
    # Calculate the coordinates for the four corners of the rectangle
    top_left = (lon - half_width, lat + half_height)
    top_right = (lon + half_width, lat + half_height)
    bottom_left = (lon - half_width, lat - half_height)
    bottom_right = (lon + half_width, lat - half_height)

    # Create a dictionary to store the rectangle data
    rectangle_data = {
        "Rectangle Number": i,
        "Center Coordinate": (lon, lat),
        "Top Left Corner": top_left,
        "Top Right Corner": top_right,
        "Bottom Left Corner": bottom_left,
        "Bottom Right Corner": bottom_right,
    }

    # Add the rectangle data to the list
    rectangle_data_list.append(rectangle_data)

# Print the list of rectangle data
    for key, value in rectangle_data.items():
        print(f"{key}: {value}")
    print()










Rectangle Number: 1
Center Coordinate: (13.45555639594199, -54.44045226080955)
Top Left Corner: (2.3055563959419896, -43.29045226080955)
Top Right Corner: (24.60555639594199, -43.29045226080955)
Bottom Left Corner: (2.3055563959419896, -65.59045226080956)
Bottom Right Corner: (24.60555639594199, -65.59045226080956)

Rectangle Number: 2
Center Coordinate: (36.14676818709064, -43.180266907414826)
Top Left Corner: (24.996768187090645, -32.03026690741483)
Top Right Corner: (47.29676818709064, -32.03026690741483)
Bottom Left Corner: (24.996768187090645, -54.330266907414824)
Bottom Right Corner: (47.29676818709064, -54.330266907414824)

Rectangle Number: 3
Center Coordinate: (42.99905927897668, -42.1325837805427)
Top Left Corner: (31.849059278976682, -30.982583780542704)
Top Right Corner: (54.14905927897668, -30.982583780542704)
Bottom Left Corner: (31.849059278976682, -53.2825837805427)
Bottom Right Corner: (54.14905927897668, -53.2825837805427)

Rectangle Number: 4
Center Coordinate: (54.9

# Defining Features

Loading our features dataset from: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_HARMONIZED#bands


In [9]:
# Initialize an empty list to hold the images
image_list = []

# Loop over each year from 2016 to 2021
for year in range(int(i_date), int(f_date)):
    start_date = ee.Date(f'{year}-01-01')
    end_date = start_date.advance(180, 'day') # Threshold 180 days to find an appropriate match to filter conditions

    filtered_images = (ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
                      .filterDate(start_date, end_date) # applying the date range
                      .filterBounds(square) # applying the bounds of the coordinates
                      .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 50) # applying cloud filter on image property
                      .sort('system:time_start')) # sorting collected images by system time

    clipped_images = filtered_images.map(clip_img) # clipping to the bounds. necessary as filtering by bounds does not clip!
    first_image = clipped_images.first() # getting the individual image, so that it can be bundled in an ImageCollection later

    if first_image:
        image_list.append(first_image)

features = ee.ImageCollection.fromImages(image_list) # create the ImageCollection


NameError: ignored

In [ ]:
# Get info about the resulting collection - how many images are in the collection? Should return 6
features.size()

# Get full depth Features nd array

## RGB + NIR bands together

In [ ]:
# Get feature bands. Because they have different resolutions we'll stick just
# to those that have resolution of 10m. If we include ones that have 20m or
# 60m resolution, then we'd have to add padding or similar to the nd array in
# its depth. B2 is blue, B3 is green, B4 is red, B8 is NIR (near infra red)
feature_bands = ["B2", "B3", "B4", "B8"]

In [ ]:
# Initialize an empty list to collect the numpy arrays
stacked_feature_list = []

# Loop through the image collection
for image in features.getInfo()['features']: # if you do .getInfo() on an image you get a dict, which we're accessing below
    image_id = image['id']
    ee_image = ee.Image(image_id)

    # Download the image as a NumPy array
    url = ee_image.getDownloadUrl({
        'bands': feature_bands,
        'region': square,
        'scale': 10,
        'format': 'NPY' # numpy
    })
    image_array = requests.get(url)
    image_array = np.load(io.BytesIO(image_array.content))

    # Append the numpy array to the list
    stacked_feature_list.append(image_array)

# Stack the arrays depth-wise
feature_stacked_array = np.stack(stacked_feature_list, axis=-1) # create a nd array where the depth dimension is time

## NDVI band only

In [ ]:
# Creating the NDVI array - NDVI is an index used for detecting forest in the academic literature
# Assuming data_array is your original array of shape (51, 51, 6) and dtype([('B2', 'f4'), ('B3', 'f4'), ('B4', 'f4'), ('B8', 'f4')])

# Extract B4 (Red) and B8 (NIR)
B4 = feature_stacked_array['B4']
B8 = feature_stacked_array['B8']

# Calculate NDVI - basically the normalised difference between Red and NIR bands
NDVI = (B8 - B4) / (B8 + B4 + 1e-10)  # adding a small constant to avoid division by zero

# Result will be a 3D ndarray of shape (51, 51, 6)

# NEW BASELINE PREDICTION FROM NDVI THRESHOLD
Any pixel with an NDVI value greater than 0.6 is classified as forest. According to ChatGPT this is typical for classifying forest (though the threshold for just classifying vegetation alone, maybe lone trees etc, is lower at 0.2-0.5)

In [ ]:
mask_ndvi = NDVI >= 0.6
NDVI_bucketed = np.where(mask_ndvi, 1, 0)

Need to apply a majority pooling to this NDVI nd array to make it compatible with the target

In [ ]:
NDVI_pooled = majority_pool(NDVI_bucketed)

# Defining Target

Loading our target dataset from: https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MCD12Q1#bands%5Blink

In [ ]:
# Filtering down for the target band and the date range
target = (ee.ImageCollection("MODIS/061/MCD12Q1").filterDate(i_date, f_date)
          .filterBounds(square)
          .sort('system:time_start'))  # Sort by time to get earliest image

In [ ]:
# Checking how many images are contained in the Image Collection, should return 6
target.size()

In [ ]:
# Apply the clipping
target = target.map(clip_img)

# Get full depth Target nd array

In [ ]:
# Get the target bands. We are only picking one, though there's several to choose from
# We'll pick the LC_Type1 just because it's first
target_bands = ["LC_Type1"]

In [ ]:
# Initialize an empty list to collect the numpy arrays
stacked_target_list = []

# Loop through the image collection
for image in target.getInfo()['features']:
    image_id = image['id']
    ee_image = ee.Image(image_id)

    # Download the image as a NumPy array
    url = ee_image.getDownloadUrl({
        'bands': target_bands,
        'region': square,
        'scale': 500, # again, notice the scale! 500m x 500m for a target pixel
        'format': 'NPY'
    })
    image_array = requests.get(url)
    image_array = np.load(io.BytesIO(image_array.content))

    # Append the numpy array to the list
    stacked_target_list.append(image_array)

# Stack the arrays depth-wise
target_stacked_array = np.stack(stacked_target_list, axis=-1)

# NEW TARGET LABELLING BUCKETING FUNCTION

Take any value in the full nd array and allocate it to buckets of "1" forest and "0" not forest. The existing labels in the target nd array are mapped as follows:

*   label -> bucket
*   1-5 -> 1
*   6-17 -> 0

In [ ]:
mask = target_stacked_array["LC_Type1"] >= 6  # Create a boolean mask where values greater than 6 are True
target_array_bucketed = np.where(mask, 0, 1)  # Assign 0 where mask is True, 1 where mask is False


# NEW ASSESS BASELINE MODEL PERFORMANCE
Compare the target classification (ground truth) against the prediction using F1, to balance precision and recall.

In [ ]:
# Reshape your arrays into 1D arrays
true_values_1D = target_array_bucketed.reshape(-1)
pred_values_1D = NDVI_pooled.reshape(-1)

# Calculate F1 score
f1 = f1_score(true_values_1D, pred_values_1D)

print("F1 Score:", f1)


F1 Score: 0.6666666666666666
